In [8]:
#importing packages
import numpy as np
import pandas as pd
import matplotlib as plt


In [9]:
#c# This script is to test the IPF (iterative proportional fitting) algortihm first with random data entries (synthetic data) and then on the mouse brain data
# the idea is to understand IPF and to have the code ready to apply it to new data

## IPF --> in the TOMOSEQ-folder there will be a tePt file ePplaining in words what the IPF is and how it works. 



#----------------------------------------------------------------------------------------------------------------#



# 
# reating synthetic dataset (just random numbers as entries and random marginals/target values) in 2D !!!

P = np.array([
    [10,20,30,5],
    [5,10,15,50],
    [30,40,20,20],
    [10,5,15,30]
    ])


u = np.array([150, 300, 400, 150]) # row target ---> make sure that these are definetely larger than the sum, when creating sythetic data !
v = np.array([200, 300, 400, 100]) # col target

print(f'observed sum of rows: {P.sum(axis=1)}')
print(f'target sum row: {u}')

print('-------- ----- ----')

print(f'observed sum of columns: {P.sum(axis=0)}')
print(f'target sum columns: {v}')

def ipf_update(M, u, v): # M = current matrix, u = tarfet sum of rows, v = target sum of columns
    r_sums = M.sum(axis=1) # calculating sum of each row (current row sum)

    #constructing new matrix N:
    # procedure: Rows first: scaling each matrix element with a factor so that the sum adds up to the target sum
    # therefore iterating over row elements and column elements to calculate the sum for all rows.

    N = np.array([[M[r,c] * u[r] / r_sums[r] for c in range(M.shape[1])] # current matrix element * factor ( ratio of target row sum and current row sum --> scales each element to match target row sum)
                    for r in range(M.shape[0])]) # need to iterate over columns to calculate rows, shape[0] is the dimension.
    r_sums[r_sums == 0] = 1  # prevent division by zero

# same for columns now ---> sum of column elements

    c_sums = N.sum(axis=0) # summing over column elements
    O = np.array([[N[r, c] * v[c] / c_sums[c] for c in range(N.shape[1])]
                    for r in range(N.shape[0])])
    c_sums[c_sums == 0] = 1  # Prevent division by zero

    d_u = np.linalg.norm(u - O.sum(axis=1), 2)
    d_v = np.linalg.norm(v - O.sum(axis=0), 2)

    return O, d_u, d_v

M = P.copy() 
print(M)


for _ in range(100):
    M, d_u, d_v = ipf_update(M, u, v)
    print(f'd_u = {d_u:.5f}, d_v = {d_v:.5f}')
    if d_u <= 0.0001 and d_v <= 0.0001:
        break

print(M) # this is the matrix with the finalized columns

# control check 
print('sum of rows from matrix M:', M.sum(axis=1)) # control check 
print('sum of columns from matrix M:', M.sum(axis=0))


observed sum of rows: [ 65  80 110  60]
target sum row: [150 300 400 150]
-------- ----- ----
observed sum of columns: [ 55  75  80 105]
target sum columns: [200 300 400 100]
[[10 20 30  5]
 [ 5 10 15 50]
 [30 40 20 20]
 [10  5 15 30]]
d_u = 112.30252, d_v = 0.00000
d_u = 13.17068, d_v = 0.00000
d_u = 1.59298, d_v = 0.00000
d_u = 0.19318, d_v = 0.00000
d_u = 0.02339, d_v = 0.00000
d_u = 0.00283, d_v = 0.00000
d_u = 0.00034, d_v = 0.00000
d_u = 0.00004, d_v = 0.00000
[[ 19.90472035  43.93303567  84.21159803   1.95064603]
 [ 31.92207315  70.45733643 135.05383369  62.56673218]
 [112.92184196 166.15821853 106.16497656  14.75499541]
 [ 35.25136454  19.45140937  74.56959173  20.72762637]]
sum of rows from matrix M: [150.00000009 299.99997545 400.00003245 149.999992  ]
sum of columns from matrix M: [200. 300. 400. 100.]


In [5]:



#### Same in 3D ####

X = np.array([
    [[10, 20, 30], [5, 10, 15], [30, 40, 20]],
    [[10, 5, 15], [20, 30, 40], [50, 60, 70]],
    [[15, 25, 35], [10, 20, 30], [40, 50, 60]]
])


# Update the target arrays to match the new dimensions
u = np.array([800, 700, 400])  # Adjusted for 3D
v = np.array([950, 650, 480])  # Adjusted for 3D

k = np.array([400, 500, 350])  # New target for the third dimension


# 3d !!!
def ipf_update_3d(M, u, v, k):
    r_sums = M.sum(axis=1)
    N = np.array([[M[r,c] * u[r] / r_sums[r] for c in range(M.shape[1])]
                    for r in range(M.shape[0])])
    r_sums[r_sums == 0] = 1  # Prevent division by zero


    c_sums = N.sum(axis=0)
    O = np.array([[N[r, c] * v[c] / c_sums[c] for c in range(N.shape[1])]
                    for r in range(N.shape[0])])
    c_sums[c_sums == 0] = 1  # Prevent division by zero

    k_sums = O.sum(axis=2)
    K = np.array([[O[r, c] * v[c] / c_sums[c] for c in range(O.shape[1])]
                    for r in range(O.shape[0])])
    k_sums[c_sums == 0] = 1  # Prevent division by zero

    d_u = np.linalg.norm(u - O.sum(axis=1), 2)
    d_v = np.linalg.norm(v - O.sum(axis=0), 2)
    d_k = np.linalg.norm(k - O.sum(axis=2), 2)

    return O, d_u, d_v,d_k

M = X.copy() 


# Update the loop to include the new distance
for _ in range(100):
    M, d_u, d_v, d_k = ipf_update_3d(M, u, v, k)
    print(f'd_u = {d_u:.5f}, d_v = {d_v:.5f}, d_k = {d_k:.5f}')
    if d_u <= 0.0001 and d_v <= 0.0001 and d_k <= 0.0001:
        break


print(M) # this is the matrix with the finalized columns

# control check 
print('sum of 1d from matrix M (3D):', M.sum(axis=0)) # control check 
print('sum of 2d from matrix M (3D):', M.sum(axis=1))
print('sum of 3d from matrix M (3D):', M.sum(axis=2))




d_u = 694.77158, d_v = 582.92367, d_k = 1335.27747
d_u = 639.01751, d_v = 582.92367, d_k = 1279.96696
d_u = 635.23533, d_v = 582.92367, d_k = 1269.28686
d_u = 634.83109, d_v = 582.92367, d_k = 1267.51446
d_u = 634.77857, d_v = 582.92367, d_k = 1267.20241
d_u = 634.77141, d_v = 582.92367, d_k = 1267.14518
d_u = 634.77041, d_v = 582.92367, d_k = 1267.13435
d_u = 634.77027, d_v = 582.92367, d_k = 1267.13225
d_u = 634.77025, d_v = 582.92367, d_k = 1267.13184
d_u = 634.77025, d_v = 582.92367, d_k = 1267.13176
d_u = 634.77025, d_v = 582.92367, d_k = 1267.13174
d_u = 634.77025, d_v = 582.92367, d_k = 1267.13174
d_u = 634.77025, d_v = 582.92367, d_k = 1267.13173
d_u = 634.77025, d_v = 582.92367, d_k = 1267.13173
d_u = 634.77025, d_v = 582.92367, d_k = 1267.13173
d_u = 634.77025, d_v = 582.92367, d_k = 1267.13173
d_u = 634.77025, d_v = 582.92367, d_k = 1267.13173
d_u = 634.77025, d_v = 582.92367, d_k = 1267.13173
d_u = 634.77025, d_v = 582.92367, d_k = 1267.13173
d_u = 634.77025, d_v = 582.9236

In [111]:
## Installing ipfn from: https://github.com/Dirguis/ipfn?tab=readme-ov-file

from ipfn.ipfn import ipfn

# Define a synthetic 3D matrix
matrix = np.array([
    [[1.0, 2.0], [3.0, 4.0]],
    [[5.0, 6.0], [7.0, 8.0]]
])

# Define the target aggregates (marginals)
agg_dim0 = np.array([10.0, 30.0])  # Sum along axis 0
agg_dim1 = np.array([20.0, 20.0])  # Sum along axis 1
agg_dim2 = np.array([16.0, 24.0])  # Sum along axis 2

# Combine aggregates and specify dimensions
aggregates = [agg_dim0, agg_dim1, agg_dim2]
dimensions = [[0], [1], [2]]

# Create the IPF object
ipf = ipfn(matrix, aggregates, dimensions)

# Perform the iteration
result = ipf.iteration()
print('- - - - - ')
print('Result for a 3D matrix:', result)




- - - - - 
Result for a 3D matrix: [[[1.1901104  2.83920507]
  [2.30486094 3.66575114]]

 [[6.56851513 9.4021671 ]
  [5.93651353 8.09287669]]]


In [186]:
## Installing ipfn from: https://github.com/Dirguis/ipfn?tab=readme-ov-file

from ipfn.ipfn import ipfn

# Define a synthetic 3D matrix
K = np.ones(shape=(3,3,3))

# Define the target aggregates (marginals)
agg_dim0 = np.array([7.0, 3.0, 20.0])  # Sum along axis 0
agg_dim1 = np.array([5.0, 15.0, 10.0])  # Sum along axis 1
agg_dim2 = np.array([16.0, 4.0, 10.0])  # Sum along axis 2

# Combine aggregates and specify dimensions
aggregates = [agg_dim0, agg_dim1, agg_dim2]
dimensions = [[0], [1], [2]]

# Create the IPF object
ipf = ipfn(K, aggregates, dimensions, convergence_rate=1e-3, max_iteration=1000)

# Perform the iteration
result = ipf.iteration()
print('- - -  - - - - - - - - ')
print('Result for a 3D matrix:', result)
print('- - - - - - - - - - - - - - - - - - - - ')


# test which part is which axis

print(result[:,0])
print('- - - - ')
print(result[:,1])
print('- - - - ')
print(result[:,2])

# Marginals along axis = 1 

Sx1 = 0.62222222 + 0.15555556 + 0.38888889 + 0.26666667 + 0.06666667  + 0.16666667 + 1.77777778 + 0.44444444 + 1.11111111
Sx2 = 1.86666667+ 0.46666667+ 1.16666667 +0.8    +    0.2    +    0.5   +   5.33333333 + 1.33333333 + 3.33333333
Sx3 = 1.24444444 + 0.31111111 + 0.77777778 + 0.53333333 + 0.13333333 + 0.33333333 +3.55555556 + 0.88888889 + 2.22222222

# Marginals along axis = 0

Sy1 = 0.62222222 + 0.15555556 + 0.38888889 + 1.86666667+ 0.46666667+ 1.16666667+ 1.24444444+ 0.31111111+ 0.77777778
Sy2 = 0.26666667+ 0.06666667+ 0.16666667  +  0.8    +    0.2   +     0.5   +  0.53333333 +0.13333333+ 0.33333333
Sy3 = 1.77777778+ 0.44444444+ 1.11111111+ 5.33333333 +1.33333333 +3.33333333 +3.55555556+ 0.88888889 +2.22222222

# Marginals along axis = 2

Sz1 = 0.62222222 + 0.26666667 + 1.77777778 + 1.86666667 + 0.8    + 5.33333333 +1.24444444+ 0.53333333+3.55555556
Sz2 = 0.15555556+ 0.06666667 + 0.44444444 + 0.46666667 + 0.2 + 1.33333333 + 0.31111111 + 0.13333333 +0.88888889
Sz3 =  0.38888889 + 0.16666667 + 1.11111111 + 0.77777778 + 0.33333333 + 2.22222222 +1.16666667 + 0.5 + 3.33333333


print(f'x marginals along axis = 1: {Sx1}, {Sx2}, {Sx3}')
print(f'y marginals along axis = 0: {Sy1}, {Sy2}, {Sy3}')
print(f'z marginals along axis = 2: {Sz1}, {Sz2}, {Sz3}')



- - -  - - - - - - - - 
Result for a 3D matrix: [[[0.62222222 0.15555556 0.38888889]
  [1.86666667 0.46666667 1.16666667]
  [1.24444444 0.31111111 0.77777778]]

 [[0.26666667 0.06666667 0.16666667]
  [0.8        0.2        0.5       ]
  [0.53333333 0.13333333 0.33333333]]

 [[1.77777778 0.44444444 1.11111111]
  [5.33333333 1.33333333 3.33333333]
  [3.55555556 0.88888889 2.22222222]]]
- - - - - - - - - - - - - - - - - - - - 
[[0.62222222 0.15555556 0.38888889]
 [0.26666667 0.06666667 0.16666667]
 [1.77777778 0.44444444 1.11111111]]
- - - - 
[[1.86666667 0.46666667 1.16666667]
 [0.8        0.2        0.5       ]
 [5.33333333 1.33333333 3.33333333]]
- - - - 
[[1.24444444 0.31111111 0.77777778]
 [0.53333333 0.13333333 0.33333333]
 [3.55555556 0.88888889 2.22222222]]
x marginals along axis = 1: 5.00000001, 15.0, 9.99999999
y marginals along axis = 0: 7.000000010000001, 3.0, 19.999999990000003
z marginals along axis = 2: 16.0, 4.0, 10.0
